# <span style="color: royalblue;">Libraries</span> 

In [ ]:
# Data handling
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# AI modeling
from keras import models
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.utils import to_categorical

In [ ]:
# Image handling
from PIL import Image

# <span style="color: royalblue;">Path</span> 

In [ ]:
path = '/kaggle/input/challenges-in-representation-learning-facial-expression-recognition-challenge/'

# <span style="color: royalblue;">Load Data</span> 

In [ ]:
data = pd.read_csv(path+'icml_face_data.csv')

In [ ]:
test_imgs = '/kaggle/input/facial-expression-sample/'
imglist = ['sample_01.jpg', 'sample_02.jpg', 'sample_03.jpg', 'sample_04.jpg', 'sample_05.jpg', 'sample_06.jpg', 'sample_07.jpg', 'sample_08.jpg', 'sample_09.jpg', 'sample_10.jpg']

# <span style="color: royalblue;">Functions</span> 

In [ ]:
def prepare_img(pic):
    """ input : 이미지 주소
        output : 48*48 흑백으로 변환된 픽셀 데이터 """
    im = Image.open(pic)

    pix = np.array(im)
    ysize=im.size[0]
    xsize=im.size[1]
    graypix=np.zeros(shape=(xsize,ysize))
    if pix.ndim == 3:
        for i in range(xsize):
            for j in range(ysize):
                p=int((0.2126*pix[i][j][0])+(0.7152*pix[i][j][1])+(0.0722*pix[i][j][2]))
                graypix[i][j]=p
    else:
        for i in range(xsize):
            for j in range(ysize):
                graypix[i][j] = pix[i][j]

    sizecheck=[[0],[0]]
    resized=str()
    for i in range(48):
        a=int(xsize*(i+1)/48)
        sizecheck[0].append(a)
    for j in range(48):        
        b=int(ysize*(j+1)/48)
        sizecheck[1].append(b)

    for i in range(48):
        for j in range(48):
            a=sizecheck[0][i+1]-sizecheck[0][i]
            b=sizecheck[1][i+1]-sizecheck[1][i]
            new=0
            for k in range(a):
                for l in range(b):
                    new+=graypix[sizecheck[0][i]+k][sizecheck[1][j]+l]
            new=str(int(new/(a*b)))
            if resized=='':
                resized=new
            else:
                resized=resized+' '+new
    return resized

In [ ]:
def prepare_data(data):
    """ input: 레이블과 픽셀 데이터(48*48)
        output: 레이블과 행렬 형태의 이미지 """
    
    image_array = np.zeros(shape=(len(data), 48, 48))
    image_label = np.array(list(map(int, data['emotion'])))
    
    for i, row in enumerate(data.index):
        image = np.fromstring(data.loc[row, ' pixels'], dtype=int, sep=' ')
        image = np.reshape(image, (48, 48))
        image_array[i] = image
        
    return image_array, image_label

# <span style="color: royalblue;">불러온 이미지 데이터의 형태 변경</span>

In [ ]:
img_data = []
for i in imglist:
    img_data.append(prepare_img(test_imgs+i))

In [ ]:
test_image_array = np.zeros(shape=(len(imglist), 48, 48))
for i in range(len(imglist)):
        image = np.fromstring(img_data[i], dtype=int, sep=' ')
        image = np.reshape(image, (48, 48))
        test_image_array[i] = image

# <span style="color: royalblue;">데이터 정리</span> 

In [ ]:
emotions = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Sad', 5: 'Surprise', 6: 'Neutral'}

* 학습용, 테스트용 데이터 분리

In [ ]:
train_image_array, train_image_label = prepare_data(data[data[' Usage']=='Training'])
val_image_array, val_image_label = prepare_data(data[data[' Usage']=='PrivateTest'])

* 픽셀 데이터를 48x48의 행렬로 변환

In [ ]:
train_images = train_image_array.reshape((train_image_array.shape[0], 48, 48, 1))
train_images = train_images.astype('float32')/255
val_images = val_image_array.reshape((val_image_array.shape[0], 48, 48, 1))
val_images = val_images.astype('float32')/255
test_images = test_image_array.reshape((test_image_array.shape[0], 48, 48, 1))
test_images = test_images.astype('float32')/255

In [ ]:
train_labels = to_categorical(train_image_label)
val_labels = to_categorical(val_image_label)

# <span style="color: royalblue;">Model</span> 
CNN 모델 정의

In [ ]:
model = models.Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(MaxPool2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPool2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(7, activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# 학습
history = model.fit(train_images, train_labels,
                    validation_data=(val_images, val_labels),
                    epochs=15,
                    batch_size=64)

* 예측

In [ ]:
pred_test_labels = model.predict(test_images)

# <span style="color: royalblue;">분석 결과</span> 

In [ ]:
def plot_image_and_emotion(test_image_array, pred_test_labels, image_number):
    """표에 이미지와 분석 결과를 나타내는 함수"""
    
    fig, axs = plt.subplots(1, 2, figsize=(12, 6), sharey=False)
    
    bar_label = emotions.values()
    
    axs[0].imshow(test_image_array[image_number], 'gray')
    axs[0].set_title('Picture')
    
    axs[1].bar(bar_label, pred_test_labels[image_number])
    axs[1].grid()
    
    plt.show()

In [ ]:
plot_image_and_emotion(test_image_array, pred_test_labels, 0)

In [ ]:
for i in range(len(imglist)):
    plot_image_and_emotion(test_image_array, pred_test_labels, i)